<a href="https://colab.research.google.com/github/sgupta701/NEURA-does-it-all/blob/main/query_splitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving queries.jsonl to queries.jsonl


In [ ]:
!pip install transformers datasets sentencepiece


In [ ]:
import json

data = []
with open("queries.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

print(f"Total examples: {len(data)}")
print("Sample input:", data[0]['input'])
print("Sample output:", data[0]['output'])


Total examples: 1064
Sample input: Send an email to Michael at michael.smith@example.com about the sales figures, then play Hotel California by Eagles and check the weather for tomorrow.
Sample output: Send an email to Michael at michael.smith@example.com about the sales figures
play Hotel California by Eagles
check the weather for tomorrow


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "t5-small"  # or "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    # Prefix input with a task name (optional, helps T5)
    input_text = "split: " + example['input']
    target_text = example['output']

    inputs = tokenizer(input_text, max_length=128, padding='max_length', truncation=True, return_tensors='pt')
    labels = tokenizer(target_text, max_length=128, padding='max_length', truncation=True, return_tensors='pt')

    # Important: replace padding token id's in labels by -100 so they're ignored by loss
    labels_input_ids = labels['input_ids']
    labels_input_ids[labels_input_ids == tokenizer.pad_token_id] = -100

    return inputs['input_ids'].squeeze(), inputs['attention_mask'].squeeze(), labels_input_ids.squeeze()


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class QuerySplitterDataset(Dataset):
    def __init__(self, data_list):
        self.data = data_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_ids, attention_mask, labels = tokenize_function(item)
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

# Split your dataset into train and validation (say 90% train, 10% val)
train_size = int(0.9 * len(data))
train_data = data[:train_size]
val_data = data[train_size:]

train_dataset = QuerySplitterDataset(train_data)
val_dataset = QuerySplitterDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)


In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)


In [ ]:
import torch
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm

# Ensure you're using GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Optional: gradient clipping threshold
max_grad_norm = 1.0

epochs = 3

for epoch in range(epochs):
    model.train()
    total_loss = 0

    # tqdm for live progress bar
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)

    for batch in loop:
        # Move all batch tensors to device at once
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward + Backward + Optimization
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"]
        )
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()

        # Optional: Clip gradients to prevent exploding gradients
        clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()

        total_loss += loss.item()

        # Update tqdm display
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch + 1} completed | Average Loss: {avg_loss:.4f}")


Epoch 1:   0%|          | 0/240 [00:00<?, ?it/s]Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


✅ Epoch 1 completed | Average Loss: 0.1466


✅ Epoch 2 completed | Average Loss: 0.0144


✅ Epoch 3 completed | Average Loss: 0.0102


In [ ]:
model.save_pretrained("finetuned_query_splitter")
tokenizer.save_pretrained("finetuned_query_splitter")


('finetuned_query_splitter/tokenizer_config.json',
 'finetuned_query_splitter/special_tokens_map.json',
 'finetuned_query_splitter/spiece.model',
 'finetuned_query_splitter/added_tokens.json',
 'finetuned_query_splitter/tokenizer.json')

In [ ]:
!zip -r finetuned_query_splitter.zip finetuned_query_splitter


  adding: finetuned_query_splitter/ (stored 0%)
  adding: finetuned_query_splitter/tokenizer.json (deflated 74%)
  adding: finetuned_query_splitter/special_tokens_map.json (deflated 85%)
  adding: finetuned_query_splitter/tokenizer_config.json (deflated 95%)
  adding: finetuned_query_splitter/generation_config.json (deflated 29%)
  adding: finetuned_query_splitter/model.safetensors (deflated 12%)
  adding: finetuned_query_splitter/spiece.model (deflated 48%)
  adding: finetuned_query_splitter/config.json (deflated 62%)


In [ ]:
from google.colab import files
files.download("finetuned_query_splitter.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>